# Script pour enregister la performance de 3 modèles
Source: https://medium.com/@www.seymour/training-an-ai-to-play-a-game-using-deep-reinforcement-learning-b63534cfdecd
Le script enregistre une vidéo dans le dossier parent des modèles

### Imports

In [3]:
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import dqn, game, record

import importlib
importlib.reload(game)
importlib.reload(dqn)

import pygame

### Modèles à charger

In [4]:
# Uniques paramètres à modifier si besoin
models = [0, 30, 100]
general_folder = "../simulations/2024-04-19 0202"

folder = os.path.join(general_folder, "models")

agents = []
envs = []
states = []
scores = [0 for m in models]

for model in models:
    env = game.Environment()
    agent = dqn.DQN(
        state_shape=env.ENVIRONMENT_SHAPE,
        action_size=env.ACTION_SPACE_SIZE
    )
    agent.load(f'{folder}/{model}.h5')

    state = env.reset()
    state = np.expand_dims(state, axis=0)

    agents.append(agent)
    envs.append(env)
    states.append(state)

padding = 10

env = envs[0]

In [5]:

pygame.init()
screen = pygame.display.set_mode((env.WINDOW_WIDTH * len(models) + (padding * (len(models) - 1)), env.WINDOW_HEIGHT))
pygame.display.set_caption(f"Models: {models}")
font = pygame.font.SysFont(None, 36)
clock = pygame.time.Clock()
running = True

record_video = True

recorder = None
if record_video:
    file_name = os.path.join(general_folder, 
                             f"comparison_{'_'.join(map(str, models))}.avi")
    recorder = record.ScreenRecorder(env.WINDOW_WIDTH * len(models) + (padding * (len(models) - 1)), env.WINDOW_HEIGHT, 15, file_name)

MAX_TICKS = 2000

tick = 0
while running:

    screen.fill("white")

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    for m, model in enumerate(models):

        x_offset = m * (env.WINDOW_WIDTH + padding)

        action = agents[m].act(states[m], 0)
        state, reward, done, scores[m] = envs[m].step(action)
        states[m] = np.expand_dims(state, axis=0)

        envs[m].render(screen, solo=False, x_offset=x_offset)

        text_surface = font.render(f"{scores[m]}", True, (50, 50, 50))
        screen.blit(text_surface, (int(x_offset + env.WINDOW_WIDTH / 2) - 10, 20))

    # black bars
    for i in range(1, len(models)):
        x_offset = i * (env.WINDOW_WIDTH + padding)
        pygame.draw.rect(
            screen,
            "black",
            ((x_offset + padding, 0), (padding, env.WINDOW_HEIGHT))
        )
    
    pygame.display.flip()
    clock.tick(30)

    if record_video:
        recorder.capture_frame(screen)

    tick += 1

    if tick >= MAX_TICKS:
        running = False

pygame.quit()
if record_video:
    recorder.end_recording()

Initializing ScreenRecorder with parameters width:1100 height:360 fps:15.
Output of the screen recording saved to ../simulations/2024-04-19 0202\comparison_0_30_100.avi.
